In [ ]:
from selenium import webdriver
import pandas as pd
import datetime, time


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait   # 해당 태그를 기다림
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
#chromedriver 다운받고, 그 파일을 이 파일과 같은 위치에 복붙해와! 경로 외우기 귀찮은관계로,,
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')    # GPU 사용 안함
options.add_argument('lang=ko_KR')    # 언어 설정
driver = webdriver.Chrome(options=options)

driver.implicitly_wait(10)


In [ ]:
import requests
from bs4 import BeautifulSoup

webpage = requests.get("https://brunch.co.kr/keyword/%EC%A7%80%EA%B5%AC%ED%95%9C%EB%B0%94%ED%80%B4_%EC%84%B8%EA%B3%84%EC%97%AC%ED%96%89?q=g")
soup = BeautifulSoup(webpage.content, "html.parser")
writers=[]
url = 'https://brunch.co.kr'
essay = soup.find_all(attrs={'class':'keyword_elem'})
for i in essay:
    key = i['href']
    driver.get(url+key)
    writer = driver.find_elements(By.CLASS_NAME, 'item_recommend')
    # soup2 = BeautifulSoup(webpage2.content, "html.parser")
    # soup2.find_all(attrs={'class':'item_recommend'})
    # writer = soup2.find_all(attrs={'class':'item_recommend'})
    # print(writer)
    for w in writer:
        ww = w.get_attribute('href')
        if ww not in writers:
            writers.append(ww)





In [ ]:
def crawling(url):
    try:
        # url = "https://brunch.co.kr"+writer_id
        raw = requests.get(url)
        html = BeautifulSoup(raw.content, "html.parser")

        post_text = html.find_all(attrs={'class':'wrap_item item_type_text'})

        total = ""
        # 머리말을 제거하기 위해 index를 3부터 시작
        for j in range(len(post_text)):
            processed = (post_text[j].text).strip()
            processed = processed.replace('<', '').replace('^','').replace('>','').replace('#','').replace('/','').replace('\n','')

            # 머리말은 continue, 꼬리말은 break로 처리       
            if "수필작가, 자유기고가" in processed:
                break
            elif "작성일" in processed:
                continue
            elif "어느 날의 글" in processed:
                continue
            elif ".com" in processed:
                continue
            elif processed == "You can make anythingby writing":
                continue
            elif processed == "- C.S.Lewis -":
                continue
            elif processed == "":
                continue
            else:
                total = total + processed + " "
        
        if total != "":
            if "BC" in total:
                return
            return total
        
    except:
        return

In [ ]:
articles = []
url = 'https://brunch.co.kr'

for writer in writers:
    driver.get(writer+'#articles')
    a = driver.find_elements(By.CLASS_NAME,'link_post.has_image.\#post_listview')
    for u in a:
        url2 = u.get_attribute('href')
        p = crawling(url2)

        try:
            f = open(".\\BLOG\\brunch_n\\"+writer[-3:]+url2[-3:]+".txt", "w", encoding = "utf-8")
            f.write("%s" % p)
            f.close()

        except:
            continue
